In [25]:
import numpy as np
import pandas as pd
import networkx as nx

In [26]:
fp=(eval(input('请输入旗点纵坐标（从0开始）：')), eval(input('请输入旗点横坐标（从0开始）：')))

mat=pd.read_excel(r'C:/Anaconda/JupyterCode/色彩翻翻求解器/3-7 (3,2).xlsx', header=None)
sl=np.shape(mat)[0]

mat

请输入旗点纵坐标（从0开始）：3
请输入旗点横坐标（从0开始）：2


,0,1,2,3,4,5,6,7,8,9,10
0,r,o,g,g,p,p,p,y,g,p,p
1,p,p,g,b,p,p,r,r,o,y,y
2,g,o,o,r,r,o,g,b,g,b,r
3,y,y,p,g,r,o,o,g,g,g,y
4,b,y,o,o,y,y,p,g,g,o,g
5,g,y,r,p,g,o,o,r,g,y,y
6,b,g,b,p,p,r,p,o,o,o,y
7,p,g,b,b,g,y,p,p,b,r,g
8,r,o,b,r,p,y,y,y,b,b,p
9,g,o,p,b,p,y,y,g,r,g,b


In [27]:
G=nx.Graph()

# 添加节点，为其设置颜色属性
for i in range(sl):
    for j in range(sl):
        G.add_node((i,j), color=mat.iloc[i,j])

# 添加边，设置边权
for i in range(sl):
    for j in range(sl):
        if j+1<=sl-1:
            G.add_edge((i,j), (i,j+1), weight=1)
        if i+1<=sl-1:
            G.add_edge((i,j), (i+1,j), weight=1)

# print(G._node)
# print(pd.DataFrame(G.edges(data=True)))

In [28]:
def merge(G, fp):
    G1=G.copy()
    for node in list(G1._node):
        if node!=fp:
            for neighbor in G1[node]:
                if (G1._node[node])['color']==(G1._node[neighbor])['color']:
                    target_neighbor=neighbor
                    other_neighbors=list(set(G1[node])-{target_neighbor})
                        
                    for curr_neighbor in other_neighbors:
                        G1.add_edge(curr_neighbor, target_neighbor, weight=1)
                            
                    G1.remove_node(node)
                    break
                    
    return G1

In [29]:
G=merge(G, fp)

# print(G._node)
# print(pd.DataFrame(G.edges(data=True)))

In [30]:
def calc_TWD(G, fp, dwc=1):
    TWD=0
    for node in G._node:
        depth=nx.dijkstra_path_length(G, source=fp, target=node)
        TWD+=depth*dwc**depth
        
    return TWD

In [31]:
def greedy_algorithm(G, fp, dwc=1):
    G1=G.copy()
    pre_TWD=calc_TWD(G1, fp, dwc)
    seq_opt=[]
    
    while pre_TWD>0:
        o_T_df=pd.DataFrame([list({'r','o','y','g','b','p'}-{(G1._node[fp])['color']}), np.empty(5, dtype=int)], \
                            index=['operation', 'TWD']).T
        for i in range(5):
            G2=G1.copy()
            operation=o_T_df.iloc[i, 0]
            (G2._node[fp])['color']=operation
            o_T_df.iloc[i, 1]=calc_TWD(merge(G2, fp), fp, dwc)
            
        operation_opt=o_T_df.sort_values(by='TWD').iloc[0, 0]
        seq_opt.append(operation_opt)
        
        (G1._node[fp])['color']=operation_opt
        G1=merge(G1, fp)
        pre_TWD=calc_TWD(G1, fp, dwc)
        
    return len(seq_opt), seq_opt

In [32]:
steps, seq_opt=greedy_algorithm(G, fp, dwc=1)

print(steps)
print(seq_opt)

12
['o', 'y', 'o', 'g', 'o', 'b', 'r', 'g', 'p', 'y', 'b', 'o']
